In [1]:
#Base and Cleaning 
import sys
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import time
import pickle
from datetime import datetime
from datetime import date
import json
import requests
import emoji
import re
import string
import warnings
from pprint import pprint
warnings.filterwarnings("ignore")

In [2]:
# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('../datasets/muntot_sentiment.csv', converters = {'tokens': eval, 'sentiment_dict':eval})

In [4]:
df['datetime']=pd.to_datetime(df['datetime'])

In [5]:
df

,username,text,tweetcreatedts,hashtags,retweetcount,likecount,acctdesc,location,followers,totaltweets,usercreatedts,datetime,time,mentions,cleantext,tokens,sentiment_dict,sentiment_score,sentiment_rating
0,ManUtd,under way at old trafford — come on united! 🔴⚪...,2020-10-04 15:30:16,"['MUFC', 'MUNTOT']",262,1917,Official #MUFC account. @ManUtd_ES 🇪🇸 | @ManUt...,"Manchester, England",23298880,61791,2012-04-20 15:17:43,2020-10-04 15:30:00,15:30:00,NaN,under way at old trafford come on united mufc...,"[way, old, trafford, come]","{'neg': 0.0, 'neu': 0.763, 'pos': 0.237, 'comp...",0.4215,pos
1,NelsonNelli2,ready for battle.\n\n#mufc #muntot https://t.c...,2020-10-04 15:30:16,"['MUFC', 'MUNTOT']",757,0,I'm jovial and fun to be with,"Port Harcourt, Nigeria",1,45,2020-09-25 07:43:39,2020-10-04 15:30:00,15:30:00,NaN,ready for battle mufc muntot,"[ready, battle]","{'neg': 0.321, 'neu': 0.37, 'pos': 0.309, 'com...",-0.0258,neu
2,Dcfox82,cmon #mufc 3pts today please,2020-10-04 15:30:16,['mufc'],0,0,"work for nhs, Husband, dad of 3, student of li...","Derby,England",595,4873,2010-10-18 16:11:48,2020-10-04 15:30:00,15:30:00,NaN,cmon mufc today please,"[cmon, today, please]","{'neg': 0.0, 'neu': 0.566, 'pos': 0.434, 'comp...",0.3182,pos
3,lynger2000,come united!!!!!!!!!!!! 🔴🔴 #muntot #mufc,2020-10-04 15:30:16,"['MUNTOT', 'MUFC']",0,0,A Scottish lass who will always and forever be...,"Leeds, England",3337,44646,2012-11-14 20:15:43,2020-10-04 15:30:00,15:30:00,NaN,come united muntot mufc,[come],"{'neg': 0.0, 'neu': 0.517, 'pos': 0.483, 'comp...",0.4215,pos
4,URMySolskjaer,"telles and cavani now done, #mufc will have o...",2020-10-04 15:30:16,['MUFC'],432,0,NaN,NaN,43,2738,2015-11-22 17:14:36,2020-10-04 15:30:00,15:30:00,NaN,telles and cavani now done mufc will have one ...,"[telles, cavani, one, final, push, see, finall...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38696,__DANEJO,define manchester united in 1 - 3 words 📽📽 #mu...,2020-10-04 17:21:27,['MUNTOT'],215,0,ᴬʲᵉᵖᵃᵏᵒ • ᴵ'ᵐ ᴼⁿˡʸ ᴴᵘᵐᵃⁿ ☺ • ᴬᵐᵇⁱᵛᵉʳᵗ • ᴱˣᵖˡᵒ...,North 🇳🇬,9140,34660,2014-06-26 11:30:26,2020-10-04 17:21:00,17:21:00,NaN,define manchester united in words muntot,"[define, word]","{'neg': 0.0, 'neu': 0.641, 'pos': 0.359, 'comp...",0.4215,pos
38697,Chachi97357115,pogba and shaw showing they are breaking. ne...,2020-10-04 17:21:27,['MUNTOT'],0,0,NaN,NaN,1,3,2020-09-29 22:56:29,2020-10-04 17:21:00,17:21:00,NaN,pogba and shaw showing they are breaking needl...,"[pogba, shaw, show, break, needless, foul]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
38698,MiesterBob,penalty to spurs...\n\n#muntot,2020-10-04 17:21:27,['MUNTOT'],329,0,extreme,"Botswana, Gaborone",2,7,2019-02-27 08:29:10,2020-10-04 17:21:00,17:21:00,NaN,penalty to spurs muntot,[penalty],"{'neg': 0.5, 'neu': 0.5, 'pos': 0.0, 'compound...",-0.4588,neg
38699,2020Kuami,who is a better defender? \n\n1. retweet for m...,2020-10-04 17:21:27,['MUNTOT'],34,0,Neva give up bro 🙌🙌🙌🙌,"Greater Accra, Ghana",248,1789,2020-10-01 17:13:59,2020-10-04 17:21:00,17:21:00,NaN,who is a better defender retweet for maguire ...,"[well, defender, maguire, zouma]","{'neg': 0.0, 'neu': 0.541, 'pos': 0.459, 'comp...",0.7003,pos


In [8]:
data = df.tokens.to_list()

# Creating the base model

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data]

In [10]:
# Define a function to evaluate the model
def evaluator(model, dictionary=id2word, texts=data, corpus=corpus):
    base_perplexity = model.log_perplexity(corpus)
    print('\nPerplexity: ', base_perplexity) 

    # Compute Coherence Score
    coherence_model = CoherenceModel(model=model, texts=data, 
                                       dictionary=id2word, coherence='c_v')
    coherence_lda_model= coherence_model.get_coherence()
    print('\nCoherence Score: ', coherence_lda_model)

In [9]:
# Build LDA model
lda_model1_01 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=50,
                                           per_word_topics=True)


In [10]:
pprint(lda_model1_01.print_topics())

[(0,
  '0.024*"go" + 0.023*"goal" + 0.015*"game" + 0.015*"username" + 0.015*"man" + '
  '0.013*"first" + 0.012*"get" + 0.011*"league" + 0.010*"time" + '
  '0.009*"match"'),
 (1,
  '0.031*"man" + 0.027*"red" + 0.022*"martial" + 0.022*"card" + 0.017*"utd" + '
  '0.016*"get" + 0.012*"lamela" + 0.010*"ole" + 0.009*"pron" + 0.009*"old"'),
 (2,
  '0.031*"username" + 0.031*"fan" + 0.028*"right" + 0.027*"evra" + 0.025*"sky" '
  '+ 0.023*"patrice" + 0.020*"mulive" + 0.019*"play" + 0.018*"maguire" + '
  '0.016*"team"'),
 (3,
  '0.035*"telles" + 0.031*"alex" + 0.025*"username" + 0.024*"porto" + '
  '0.021*"medical" + 0.021*"hour" + 0.021*"next" + 0.019*"last" + '
  '0.019*"transfer" + 0.018*"go"')]


In [11]:
evaluator(lda_model1_01)


Perplexity:  -6.510383259543469

Coherence Score:  0.4128503545955098


In [12]:
modelseries1results = [['model1_01',4,5,50,'symmetric',-6.5080,0.4128]]


In [13]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model1_01, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.216207  0.075405       1        1  27.611710
1      0.046306 -0.147481       2        1  26.629687
0      0.010626 -0.078388       3        1  25.821261
2      0.159275  0.150464       4        1  19.937341, topic_info=         Term         Freq        Total Category  logprob  loglift
20     telles  2914.000000  2914.000000  Default  30.0000  30.0000
22       alex  2492.000000  2492.000000  Default  29.0000  29.0000
141      evra  1510.000000  1510.000000  Default  28.0000  28.0000
127       fan  2084.000000  2084.000000  Default  27.0000  27.0000
29      porto  1851.000000  1851.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
60   username  1694.883444  5266.870522   Topic4  -3.4731   0.4788
191       son   675.318317  1634.921087   Topic4  -4.3933   0.7284
18        see   632.002353  1517.362971   Topic4  -4.4595   0.7367
25         go   725.719497  3988.514670   Topic4  -4.3213  -0.0914
261       ole   440.841311  1453.540196   Topic4  -4.8198   0.4195

[305 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
47        1  0.986728     accept
47        2  0.004600     accept
47        3  0.006900     accept
47        4  0.002300     accept
48        1  0.952654        add
...     ...       ...        ...
1265      2  0.007561    yesssss
1265      3  0.077497    yesssss
1265      4  0.911060    yesssss
5415      2  0.991641  youve_sat
6725      3  0.990506  zoom_call

[754 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3])

# Hyperparameter tuning
#### Testing for optimal model number

In [14]:
lda_model1_02 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=50,
                                           per_word_topics=True)
pprint(lda_model1_02.print_topics())
evaluator(lda_model1_02)

[(0,
  '0.029*"go" + 0.016*"game" + 0.016*"first" + 0.013*"lamela" + 0.012*"pron" + '
  '0.012*"get" + 0.011*"maguire" + 0.011*"need" + 0.010*"match" + '
  '0.010*"time"'),
 (1,
  '0.047*"man" + 0.034*"red" + 0.028*"card" + 0.027*"utd" + 0.027*"martial" + '
  '0.017*"get" + 0.017*"penalty" + 0.014*"lamela" + 0.009*"pron" + '
  '0.009*"ole"'),
 (2,
  '0.036*"fan" + 0.032*"right" + 0.031*"evra" + 0.029*"sky" + 0.027*"patrice" '
  '+ 0.024*"mulive" + 0.021*"play" + 0.020*"maguire" + 0.018*"username" + '
  '0.017*"one"'),
 (3,
  '0.048*"telles" + 0.043*"alex" + 0.033*"porto" + 0.030*"hour" + '
  '0.030*"medical" + 0.030*"next" + 0.027*"transfer" + 0.026*"last" + '
  '0.026*"reach" + 0.026*"agreement"'),
 (4,
  '0.050*"username" + 0.035*"goal" + 0.026*"son" + 0.022*"man" + 0.017*"utd" + '
  '0.015*"old" + 0.013*"kane" + 0.012*"trafford" + 0.010*"mins" + '
  '0.010*"fernandes"')]

Perplexity:  -6.434963617995792

Coherence Score:  0.4701238951107715


In [15]:
modelseries1results.append(['model1_02',5,5,50,'symmetric',-6.4341,0.4701])

In [16]:
lda_model1_03 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=50,
                                           per_word_topics=True)
pprint(lda_model1_03.print_topics())
evaluator(lda_model1_03)

[(0,
  '0.035*"go" + 0.017*"first" + 0.016*"lamela" + 0.012*"game" + 0.011*"think" '
  '+ 0.011*"maguire" + 0.011*"goal" + 0.011*"need" + 0.011*"time" + '
  '0.011*"league"'),
 (1,
  '0.054*"man" + 0.036*"red" + 0.031*"card" + 0.031*"utd" + 0.029*"martial" + '
  '0.021*"penalty" + 0.017*"get" + 0.015*"lamela" + 0.010*"send" + '
  '0.009*"ole"'),
 (2,
  '0.040*"fan" + 0.036*"right" + 0.035*"evra" + 0.032*"sky" + 0.031*"patrice" '
  '+ 0.025*"mulive" + 0.024*"play" + 0.020*"maguire" + 0.019*"team" + '
  '0.018*"sancho"'),
 (3,
  '0.051*"telles" + 0.049*"alex" + 0.039*"porto" + 0.034*"medical" + '
  '0.032*"hour" + 0.031*"transfer" + 0.030*"reach" + 0.030*"next" + '
  '0.030*"agreement" + 0.030*"last"'),
 (4,
  '0.038*"goal" + 0.034*"username" + 0.030*"son" + 0.025*"man" + 0.019*"utd" + '
  '0.017*"old" + 0.015*"kane" + 0.013*"trafford" + 0.011*"mins" + '
  '0.011*"fernandes"'),
 (5,
  '0.063*"username" + 0.020*"game" + 0.015*"pron" + 0.012*"one" + 0.011*"get" '
  '+ 0.011*"sign" + 0.008*

In [17]:
modelseries1results.append(['model1_03',6,5,50,'symmetric',-6.4243,0.4459])

In [18]:
lda_model1_04 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=50,
                                           per_word_topics=True)
pprint(lda_model1_04.print_topics())
evaluator(lda_model1_04)

[(0,
  '0.040*"go" + 0.021*"first" + 0.019*"lamela" + 0.015*"goal" + 0.015*"league" '
  '+ 0.014*"need" + 0.013*"game" + 0.013*"think" + 0.012*"martial" + '
  '0.012*"let"'),
 (1,
  '0.061*"man" + 0.044*"red" + 0.037*"card" + 0.036*"martial" + 0.035*"utd" + '
  '0.024*"penalty" + 0.019*"lamela" + 0.018*"get" + 0.012*"send" + '
  '0.010*"need"'),
 (2,
  '0.029*"fan" + 0.026*"right" + 0.026*"play" + 0.025*"patrice" + 0.025*"team" '
  '+ 0.023*"evra" + 0.022*"one" + 0.022*"mulive" + 0.021*"sky" + '
  '0.021*"maguire"'),
 (3,
  '0.053*"telles" + 0.052*"alex" + 0.042*"porto" + 0.035*"medical" + '
  '0.034*"hour" + 0.033*"transfer" + 0.033*"reach" + 0.033*"next" + '
  '0.032*"agreement" + 0.032*"last"'),
 (4,
  '0.040*"goal" + 0.037*"username" + 0.032*"son" + 0.027*"man" + 0.020*"utd" + '
  '0.019*"old" + 0.015*"trafford" + 0.013*"mins" + 0.012*"fernandes" + '
  '0.012*"bruuunnooooo"'),
 (5,
  '0.076*"username" + 0.021*"game" + 0.012*"get" + 0.012*"one" + 0.012*"sign" '
  '+ 0.011*"pron" + 0

In [19]:
modelseries1results.append(['model1_04',7,5,50,'symmetric',-6.4231,0.4334])

In [20]:
lda_model1_05 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=50,
                                           per_word_topics=True)
pprint(lda_model1_05.print_topics())
evaluator(lda_model1_05)

[(0,
  '0.038*"go" + 0.022*"first" + 0.021*"lamela" + 0.015*"goal" + 0.014*"need" + '
  '0.013*"martial" + 0.013*"think" + 0.013*"game" + 0.012*"league" + '
  '0.012*"time"'),
 (1,
  '0.067*"man" + 0.048*"red" + 0.041*"card" + 0.038*"utd" + 0.037*"martial" + '
  '0.021*"penalty" + 0.020*"get" + 0.017*"lamela" + 0.011*"send" + '
  '0.010*"need"'),
 (2,
  '0.032*"fan" + 0.027*"play" + 0.026*"right" + 0.026*"patrice" + 0.026*"team" '
  '+ 0.024*"evra" + 0.024*"one" + 0.023*"maguire" + 0.022*"sky" + '
  '0.022*"deserve"'),
 (3,
  '0.036*"live" + 0.026*"telles" + 0.025*"alex" + 0.024*"stream" + '
  '0.024*"penalty" + 0.023*"fee" + 0.022*"player" + 0.021*"add" + '
  '0.020*"porto" + 0.020*"around"'),
 (4,
  '0.044*"goal" + 0.038*"username" + 0.034*"son" + 0.028*"man" + 0.020*"utd" + '
  '0.020*"old" + 0.016*"trafford" + 0.014*"mins" + 0.013*"fernandes" + '
  '0.013*"bruuunnooooo"'),
 (5,
  '0.070*"username" + 0.023*"game" + 0.014*"one" + 0.014*"sign" + 0.012*"get" '
  '+ 0.011*"pron" + 0.009

In [21]:
modelseries1results.append(['model1_05',8,5,50,'symmetric',-6.3976,0.4446])

In [22]:
lda_model1_06 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=50,
                                           per_word_topics=True)
pprint(lda_model1_06.print_topics())
evaluator(lda_model1_06)

[(0,
  '0.044*"go" + 0.025*"first" + 0.018*"goal" + 0.016*"league" + 0.015*"time" + '
  '0.015*"need" + 0.015*"game" + 0.014*"let" + 0.013*"man" + 0.013*"match"'),
 (1,
  '0.061*"man" + 0.043*"utd" + 0.027*"penalty" + 0.023*"get" + 0.016*"red" + '
  '0.016*"card" + 0.016*"martial" + 0.010*"ole" + 0.010*"need" + 0.010*"min"'),
 (2,
  '0.031*"fan" + 0.028*"right" + 0.027*"play" + 0.027*"team" + 0.026*"patrice" '
  '+ 0.026*"maguire" + 0.024*"evra" + 0.022*"one" + 0.022*"sky" + '
  '0.022*"deserve"'),
 (3,
  '0.040*"live" + 0.031*"telles" + 0.030*"alex" + 0.027*"fee" + 0.027*"stream" '
  '+ 0.024*"add" + 0.024*"porto" + 0.023*"penalty" + 0.023*"player" + '
  '0.022*"join"'),
 (4,
  '0.048*"goal" + 0.035*"son" + 0.028*"username" + 0.027*"man" + 0.021*"utd" + '
  '0.021*"old" + 0.016*"trafford" + 0.015*"mins" + 0.014*"bruuunnooooo" + '
  '0.014*"yesssss"'),
 (5,
  '0.066*"username" + 0.027*"game" + 0.015*"one" + 0.015*"sign" + 0.012*"get" '
  '+ 0.011*"cavani" + 0.011*"pron" + 0.010*"see" +

In [23]:
modelseries1results.append(['model1_06',9,5,50,'symmetric',-6.3812,0.4531])

In [24]:
lda_model1_07 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=50,
                                           per_word_topics=True)
pprint(lda_model1_07.print_topics())
evaluator(lda_model1_07)

[(0,
  '0.045*"go" + 0.025*"first" + 0.020*"goal" + 0.018*"league" + 0.017*"game" + '
  '0.014*"let" + 0.014*"time" + 0.013*"need" + 0.013*"match" + '
  '0.013*"premier"'),
 (1,
  '0.057*"man" + 0.035*"utd" + 0.020*"get" + 0.017*"red" + 0.017*"card" + '
  '0.016*"martial" + 0.012*"penalty" + 0.011*"player" + 0.010*"min" + '
  '0.010*"need"'),
 (2,
  '0.035*"fan" + 0.031*"patrice" + 0.031*"right" + 0.031*"play" + 0.029*"evra" '
  '+ 0.028*"one" + 0.028*"team" + 0.027*"maguire" + 0.027*"sky" + '
  '0.026*"mulive"'),
 (3,
  '0.062*"penalty" + 0.052*"live" + 0.038*"man" + 0.034*"stream" + 0.032*"utd" '
  '+ 0.022*"link_gtgt" + 0.019*"league" + 0.019*"vs" + 0.016*"premier" + '
  '0.012*"free"'),
 (4,
  '0.050*"goal" + 0.039*"son" + 0.029*"man" + 0.026*"username" + 0.021*"utd" + '
  '0.021*"old" + 0.016*"trafford" + 0.016*"mins" + 0.015*"kane" + '
  '0.015*"fernandes"'),
 (5,
  '0.067*"username" + 0.022*"game" + 0.014*"get" + 0.013*"pron" + 0.013*"one" '
  '+ 0.011*"sign" + 0.009*"sancho" + 

In [25]:
modelseries1results.append(['model1_07',10,5,50,'symmetric',-6.3789,0.4203])

In [26]:
lda_model1_08 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=11, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=50,
                                           per_word_topics=True)
pprint(lda_model1_08.print_topics())
evaluator(lda_model1_08)

[(0,
  '0.048*"go" + 0.028*"first" + 0.021*"goal" + 0.019*"man" + 0.018*"need" + '
  '0.017*"league" + 0.016*"win" + 0.016*"let" + 0.015*"game" + 0.015*"match"'),
 (1,
  '0.053*"man" + 0.033*"utd" + 0.023*"get" + 0.019*"card" + 0.018*"red" + '
  '0.017*"martial" + 0.012*"ole" + 0.012*"min" + 0.012*"penalty" + '
  '0.011*"player"'),
 (2,
  '0.033*"fan" + 0.031*"patrice" + 0.029*"right" + 0.029*"play" + 0.028*"team" '
  '+ 0.028*"evra" + 0.026*"one" + 0.025*"mulive" + 0.025*"sky" + '
  '0.023*"deserve"'),
 (3,
  '0.060*"penalty" + 0.056*"live" + 0.040*"man" + 0.036*"stream" + 0.034*"utd" '
  '+ 0.023*"link_gtgt" + 0.020*"vs" + 0.019*"league" + 0.017*"premier" + '
  '0.012*"free"'),
 (4,
  '0.053*"goal" + 0.043*"son" + 0.032*"man" + 0.027*"username" + 0.024*"utd" + '
  '0.019*"old" + 0.018*"mins" + 0.016*"trafford" + 0.016*"fernandes" + '
  '0.016*"bruuunnooooo"'),
 (5,
  '0.065*"username" + 0.025*"game" + 0.013*"one" + 0.013*"get" + 0.011*"sign" '
  '+ 0.010*"pron" + 0.010*"watch" + 0.00

In [27]:
modelseries1results.append(['model1_08',11,5,50,'symmetric',-6.3670,0.4344])

In [28]:
lda_model1_09 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=50,
                                           per_word_topics=True)
pprint(lda_model1_09.print_topics())
evaluator(lda_model1_09)

[(0,
  '0.054*"go" + 0.028*"first" + 0.017*"league" + 0.017*"let" + 0.016*"game" + '
  '0.016*"goal" + 0.015*"need" + 0.015*"win" + 0.014*"half" + 0.013*"time"'),
 (1,
  '0.062*"man" + 0.038*"utd" + 0.018*"need" + 0.018*"get" + 0.017*"red" + '
  '0.017*"martial" + 0.015*"card" + 0.013*"ole" + 0.013*"player" + '
  '0.012*"min"'),
 (2,
  '0.041*"fan" + 0.038*"patrice" + 0.036*"play" + 0.035*"evra" + 0.035*"right" '
  '+ 0.034*"team" + 0.033*"one" + 0.032*"sky" + 0.031*"mulive" + '
  '0.030*"deserve"'),
 (3,
  '0.060*"penalty" + 0.047*"live" + 0.038*"man" + 0.032*"utd" + 0.031*"stream" '
  '+ 0.020*"link_gtgt" + 0.019*"league" + 0.017*"vs" + 0.014*"premier" + '
  '0.013*"match"'),
 (4,
  '0.051*"goal" + 0.041*"son" + 0.033*"username" + 0.031*"man" + 0.022*"utd" + '
  '0.020*"old" + 0.017*"mins" + 0.016*"trafford" + 0.016*"fernandes" + '
  '0.015*"bruuunnooooo"'),
 (5,
  '0.054*"username" + 0.028*"game" + 0.016*"get" + 0.015*"one" + '
  '0.012*"penalty" + 0.010*"sign" + 0.010*"pron" + 0.00

In [29]:
modelseries1results.append(['model1_09',12,5,50,'symmetric',-6.3582, 0.4226])

In [30]:
modeltracker = pd.DataFrame(modelseries1results, columns =['model_name', 'num_topics', 'passes','iterations','alpha','perplexity','coherence'], dtype = float) 

In [31]:
modeltracker

,model_name,num_topics,passes,iterations,alpha,perplexity,coherence
0,model1_01,4.0,5.0,50.0,symmetric,-6.5080,0.4128
1,model1_02,5.0,5.0,50.0,symmetric,-6.4341,0.4701
2,model1_03,6.0,5.0,50.0,symmetric,-6.4243,0.4459
3,model1_04,7.0,5.0,50.0,symmetric,-6.4231,0.4334
4,model1_05,8.0,5.0,50.0,symmetric,-6.3976,0.4446
5,model1_06,9.0,5.0,50.0,symmetric,-6.3812,0.4531
6,model1_07,10.0,5.0,50.0,symmetric,-6.3789,0.4203
7,model1_08,11.0,5.0,50.0,symmetric,-6.3670,0.4344
8,model1_09,12.0,5.0,50.0,symmetric,-6.3582,0.4226


In [32]:
lda_model2_01 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=60,
                                           per_word_topics=True)
pprint(lda_model2_01.print_topics())
evaluator(lda_model2_01)

[(0,
  '0.043*"go" + 0.025*"first" + 0.018*"goal" + 0.016*"league" + 0.015*"time" + '
  '0.015*"game" + 0.014*"need" + 0.014*"man" + 0.013*"let" + 0.013*"match"'),
 (1,
  '0.058*"man" + 0.041*"utd" + 0.025*"penalty" + 0.023*"get" + 0.014*"red" + '
  '0.014*"card" + 0.014*"martial" + 0.011*"ole" + 0.010*"need" + 0.010*"min"'),
 (2,
  '0.032*"fan" + 0.029*"right" + 0.028*"play" + 0.028*"patrice" + 0.027*"team" '
  '+ 0.025*"maguire" + 0.025*"evra" + 0.024*"one" + 0.023*"sky" + '
  '0.022*"deserve"'),
 (3,
  '0.040*"live" + 0.030*"telles" + 0.029*"alex" + 0.027*"stream" + 0.026*"fee" '
  '+ 0.026*"penalty" + 0.023*"add" + 0.023*"porto" + 0.022*"player" + '
  '0.021*"join"'),
 (4,
  '0.047*"goal" + 0.036*"son" + 0.028*"username" + 0.028*"man" + 0.021*"old" + '
  '0.021*"utd" + 0.016*"trafford" + 0.015*"mins" + 0.014*"kane" + '
  '0.013*"fernandes"'),
 (5,
  '0.065*"username" + 0.026*"game" + 0.014*"one" + 0.014*"sign" + 0.012*"get" '
  '+ 0.011*"pron" + 0.011*"cavani" + 0.010*"see" + 0.009

In [33]:
modelseries2results = [['model2_01',9,5,60,'symmetric',-6.3813,0.4580]]

In [34]:
lda_model2_02 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=70,
                                           per_word_topics=True)
pprint(lda_model2_02.print_topics())
evaluator(lda_model2_02)

[(0,
  '0.043*"go" + 0.026*"first" + 0.018*"goal" + 0.016*"league" + 0.016*"time" + '
  '0.015*"need" + 0.015*"game" + 0.014*"match" + 0.013*"man" + 0.013*"let"'),
 (1,
  '0.058*"man" + 0.041*"utd" + 0.023*"get" + 0.023*"penalty" + 0.014*"red" + '
  '0.014*"card" + 0.014*"martial" + 0.012*"ole" + 0.010*"need" + '
  '0.010*"player"'),
 (2,
  '0.033*"fan" + 0.030*"right" + 0.029*"play" + 0.029*"patrice" + 0.029*"team" '
  '+ 0.026*"evra" + 0.025*"one" + 0.025*"maguire" + 0.025*"sky" + '
  '0.024*"mulive"'),
 (3,
  '0.040*"live" + 0.029*"telles" + 0.029*"alex" + 0.027*"stream" + '
  '0.026*"penalty" + 0.026*"fee" + 0.023*"add" + 0.023*"porto" + '
  '0.022*"player" + 0.021*"join"'),
 (4,
  '0.047*"goal" + 0.035*"son" + 0.032*"username" + 0.028*"man" + 0.021*"old" + '
  '0.020*"utd" + 0.016*"trafford" + 0.015*"mins" + 0.014*"kane" + '
  '0.013*"fernandes"'),
 (5,
  '0.058*"username" + 0.026*"game" + 0.015*"one" + 0.015*"sign" + 0.012*"get" '
  '+ 0.011*"cavani" + 0.011*"pron" + 0.010*"see" 

In [35]:
modelseries2results.append(['model2_02',9,5,70,'symmetric',-6.3767,0.4574])

In [36]:
lda_model2_03 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=80,
                                           per_word_topics=True)
pprint(lda_model2_03.print_topics())
evaluator(lda_model2_03)

[(0,
  '0.043*"go" + 0.026*"first" + 0.019*"goal" + 0.017*"league" + 0.016*"time" + '
  '0.015*"game" + 0.014*"need" + 0.013*"let" + 0.013*"man" + 0.013*"match"'),
 (1,
  '0.058*"man" + 0.041*"utd" + 0.024*"penalty" + 0.024*"get" + 0.015*"red" + '
  '0.014*"card" + 0.014*"martial" + 0.011*"ole" + 0.010*"min" + 0.010*"need"'),
 (2,
  '0.033*"fan" + 0.030*"right" + 0.029*"play" + 0.028*"team" + 0.028*"patrice" '
  '+ 0.026*"evra" + 0.025*"one" + 0.024*"sky" + 0.023*"deserve" + '
  '0.023*"understand"'),
 (3,
  '0.039*"live" + 0.029*"telles" + 0.029*"alex" + 0.027*"penalty" + '
  '0.026*"stream" + 0.026*"fee" + 0.023*"add" + 0.023*"porto" + 0.022*"player" '
  '+ 0.021*"join"'),
 (4,
  '0.047*"goal" + 0.036*"son" + 0.033*"username" + 0.028*"man" + 0.021*"old" + '
  '0.021*"utd" + 0.016*"trafford" + 0.015*"mins" + 0.013*"kane" + '
  '0.013*"fernandes"'),
 (5,
  '0.058*"username" + 0.025*"game" + 0.015*"one" + 0.015*"sign" + 0.012*"get" '
  '+ 0.011*"pron" + 0.011*"cavani" + 0.010*"see" + 0.

In [37]:
modelseries2results.append(['model2_03',9,5,80,'symmetric',-6.3798,0.4481])

In [38]:
lda_model2_04 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=90,
                                           per_word_topics=True)
pprint(lda_model2_04.print_topics())
evaluator(lda_model2_04)

[(0,
  '0.042*"go" + 0.025*"first" + 0.018*"goal" + 0.016*"league" + 0.015*"time" + '
  '0.015*"game" + 0.015*"need" + 0.013*"man" + 0.013*"let" + 0.012*"match"'),
 (1,
  '0.059*"man" + 0.042*"utd" + 0.024*"penalty" + 0.023*"get" + 0.015*"red" + '
  '0.015*"card" + 0.014*"martial" + 0.011*"ole" + 0.010*"need" + 0.010*"min"'),
 (2,
  '0.031*"fan" + 0.029*"right" + 0.028*"play" + 0.027*"team" + 0.027*"patrice" '
  '+ 0.025*"maguire" + 0.024*"evra" + 0.023*"one" + 0.023*"sky" + '
  '0.022*"deserve"'),
 (3,
  '0.040*"live" + 0.030*"telles" + 0.029*"alex" + 0.027*"stream" + 0.026*"fee" '
  '+ 0.026*"penalty" + 0.024*"add" + 0.023*"porto" + 0.022*"player" + '
  '0.021*"move"'),
 (4,
  '0.048*"goal" + 0.037*"son" + 0.028*"username" + 0.027*"man" + 0.021*"utd" + '
  '0.021*"old" + 0.016*"trafford" + 0.015*"mins" + 0.014*"kane" + '
  '0.014*"bruuunnooooo"'),
 (5,
  '0.065*"username" + 0.026*"game" + 0.015*"one" + 0.014*"sign" + 0.012*"get" '
  '+ 0.011*"cavani" + 0.011*"pron" + 0.010*"see" + 0.

In [39]:
modelseries2results.append(['model2_04',9,5,90,'symmetric',-6.3827,0.4579])

In [40]:
lda_model2_05 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=5,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)
pprint(lda_model2_05.print_topics())
evaluator(lda_model2_05)

[(0,
  '0.043*"go" + 0.026*"first" + 0.019*"goal" + 0.016*"league" + 0.016*"time" + '
  '0.015*"game" + 0.014*"need" + 0.013*"let" + 0.013*"man" + 0.013*"match"'),
 (1,
  '0.059*"man" + 0.041*"utd" + 0.024*"penalty" + 0.023*"get" + 0.015*"red" + '
  '0.014*"card" + 0.014*"martial" + 0.011*"ole" + 0.010*"need" + '
  '0.010*"maguire"'),
 (2,
  '0.032*"fan" + 0.029*"right" + 0.029*"play" + 0.028*"team" + 0.028*"patrice" '
  '+ 0.025*"evra" + 0.024*"one" + 0.024*"maguire" + 0.024*"sky" + '
  '0.023*"deserve"'),
 (3,
  '0.040*"live" + 0.029*"telles" + 0.028*"alex" + 0.027*"penalty" + '
  '0.026*"stream" + 0.026*"fee" + 0.023*"add" + 0.023*"porto" + 0.022*"player" '
  '+ 0.021*"move"'),
 (4,
  '0.046*"goal" + 0.035*"son" + 0.032*"username" + 0.028*"man" + 0.020*"old" + '
  '0.020*"utd" + 0.016*"trafford" + 0.015*"mins" + 0.014*"kane" + '
  '0.013*"bruuunnooooo"'),
 (5,
  '0.058*"username" + 0.026*"game" + 0.015*"one" + 0.015*"sign" + 0.012*"get" '
  '+ 0.011*"pron" + 0.011*"cavani" + 0.010*"

In [41]:
modelseries2results.append(['model2_05',9,5,100,'symmetric',-6.3796,0.4584])

In [42]:
modelseries2results

[['model2_01', 9, 5, 60, 'symmetric', -6.3813, 0.458],
 ['model2_02', 9, 5, 70, 'symmetric', -6.3767, 0.4574],
 ['model2_03', 9, 5, 80, 'symmetric', -6.3798, 0.4481],
 ['model2_04', 9, 5, 90, 'symmetric', -6.3827, 0.4579],
 ['model2_05', 9, 5, 100, 'symmetric', -6.3796, 0.4584]]

In [43]:
modeltracker2 = pd.DataFrame(modelseries2results, columns =['model_name', 'num_topics', 'passes','iterations','alpha','perplexity','coherence'], dtype = float) 

In [44]:
modeltracker = modeltracker.append(modeltracker2, ignore_index=True)

In [45]:
lda_model3_01 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=10,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=90,
                                           per_word_topics=True)
pprint(lda_model3_01.print_topics())
evaluator(lda_model3_01)

[(0,
  '0.045*"go" + 0.030*"first" + 0.020*"goal" + 0.019*"time" + 0.018*"game" + '
  '0.017*"league" + 0.015*"need" + 0.015*"half" + 0.014*"let" + 0.014*"match"'),
 (1,
  '0.068*"man" + 0.047*"utd" + 0.040*"penalty" + 0.024*"get" + 0.015*"ole" + '
  '0.012*"maguire" + 0.010*"player" + 0.009*"min" + 0.008*"need" + '
  '0.008*"defend"'),
 (2,
  '0.039*"fan" + 0.039*"right" + 0.037*"play" + 0.034*"patrice" + 0.034*"team" '
  '+ 0.032*"evra" + 0.031*"one" + 0.029*"sky" + 0.029*"deserve" + '
  '0.028*"mulive"'),
 (3,
  '0.059*"live" + 0.036*"stream" + 0.035*"fee" + 0.032*"add" + 0.030*"telles" '
  '+ 0.030*"alex" + 0.028*"join" + 0.028*"move" + 0.028*"also" + '
  '0.028*"player"'),
 (4,
  '0.053*"goal" + 0.043*"son" + 0.030*"man" + 0.025*"old" + 0.024*"username" + '
  '0.022*"utd" + 0.019*"trafford" + 0.018*"kane" + 0.016*"mins" + '
  '0.015*"fernandes"'),
 (5,
  '0.072*"username" + 0.022*"game" + 0.014*"one" + 0.013*"get" + 0.013*"sign" '
  '+ 0.013*"cavani" + 0.012*"pron" + 0.010*"back" 

In [46]:
modelseries3results = [['model3_01',9,10,90,'symmetric',-6.2963,0.4576]]

In [11]:
lda_model3_02 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=15,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=90,
                                           per_word_topics=True)
pprint(lda_model3_02.print_topics())
evaluator(lda_model3_02)

[(0,
  '0.047*"go" + 0.031*"first" + 0.020*"time" + 0.019*"game" + 0.019*"goal" + '
  '0.017*"league" + 0.016*"half" + 0.015*"need" + 0.014*"let" + 0.013*"match"'),
 (1,
  '0.069*"man" + 0.049*"utd" + 0.043*"penalty" + 0.024*"ole" + 0.023*"get" + '
  '0.011*"maguire" + 0.010*"player" + 0.009*"min" + 0.009*"wheel" + '
  '0.008*"need"'),
 (2,
  '0.044*"right" + 0.042*"play" + 0.042*"fan" + 0.038*"patrice" + 0.037*"team" '
  '+ 0.036*"evra" + 0.035*"one" + 0.032*"deserve" + 0.032*"sky" + '
  '0.031*"mulive"'),
 (3,
  '0.064*"live" + 0.037*"stream" + 0.037*"fee" + 0.034*"add" + 0.031*"telles" '
  '+ 0.030*"alex" + 0.030*"join" + 0.030*"also" + 0.030*"player" + '
  '0.030*"move"'),
 (4,
  '0.054*"goal" + 0.043*"son" + 0.031*"man" + 0.028*"username" + 0.026*"old" + '
  '0.023*"utd" + 0.020*"trafford" + 0.020*"kane" + 0.017*"score" + '
  '0.016*"mins"'),
 (5,
  '0.066*"username" + 0.021*"game" + 0.014*"get" + 0.014*"pron" + 0.014*"one" '
  '+ 0.013*"cavani" + 0.012*"sign" + 0.010*"see" + 0.01

In [48]:
modelseries3results.append(['model3_02',9,15,90,'symmetric',-6.2641,0.4606])

In [49]:
lda_model3_03 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=20,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=90,
                                           per_word_topics=True)
pprint(lda_model3_03.print_topics())
evaluator(lda_model3_03)

[(0,
  '0.047*"go" + 0.031*"first" + 0.020*"time" + 0.019*"game" + 0.018*"goal" + '
  '0.017*"need" + 0.017*"league" + 0.016*"match" + 0.016*"half" + 0.014*"let"'),
 (1,
  '0.070*"man" + 0.051*"penalty" + 0.049*"utd" + 0.024*"get" + 0.022*"ole" + '
  '0.013*"maguire" + 0.010*"player" + 0.009*"min" + 0.008*"blame" + '
  '0.008*"need"'),
 (2,
  '0.046*"right" + 0.045*"fan" + 0.044*"play" + 0.039*"patrice" + 0.039*"team" '
  '+ 0.037*"evra" + 0.037*"one" + 0.034*"sky" + 0.034*"deserve" + '
  '0.033*"mulive"'),
 (3,
  '0.067*"live" + 0.039*"stream" + 0.038*"fee" + 0.035*"add" + 0.034*"player" '
  '+ 0.032*"around" + 0.031*"also" + 0.031*"join" + 0.031*"telles" + '
  '0.031*"alex"'),
 (4,
  '0.057*"goal" + 0.046*"son" + 0.032*"man" + 0.028*"old" + 0.023*"utd" + '
  '0.022*"username" + 0.021*"trafford" + 0.021*"kane" + 0.018*"bruno" + '
  '0.017*"score"'),
 (5,
  '0.079*"username" + 0.018*"game" + 0.015*"back" + 0.014*"pron" + 0.013*"one" '
  '+ 0.013*"get" + 0.013*"cavani" + 0.012*"sign" + 

In [50]:
modelseries3results.append(['model3_03',9,20,90,'symmetric',-6.2491,0.4612])

In [51]:
lda_model3_04 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=25,
                                           random_state=1892,
                                           alpha='symmetric',
                                           iterations=90,
                                           per_word_topics=True)
pprint(lda_model3_04.print_topics())
evaluator(lda_model3_04)

[(0,
  '0.048*"go" + 0.031*"first" + 0.023*"game" + 0.020*"time" + 0.017*"goal" + '
  '0.016*"match" + 0.016*"need" + 0.016*"league" + 0.016*"half" + 0.014*"let"'),
 (1,
  '0.068*"man" + 0.047*"penalty" + 0.047*"utd" + 0.024*"ole" + 0.022*"get" + '
  '0.017*"maguire" + 0.011*"player" + 0.009*"defend" + 0.008*"min" + '
  '0.008*"need"'),
 (2,
  '0.048*"right" + 0.046*"fan" + 0.045*"play" + 0.040*"patrice" + 0.039*"one" '
  '+ 0.038*"team" + 0.038*"evra" + 0.036*"sky" + 0.035*"deserve" + '
  '0.035*"mulive"'),
 (3,
  '0.067*"live" + 0.039*"stream" + 0.038*"fee" + 0.035*"add" + 0.034*"player" '
  '+ 0.032*"around" + 0.032*"also" + 0.031*"join" + 0.031*"telles" + '
  '0.031*"move"'),
 (4,
  '0.058*"goal" + 0.046*"son" + 0.032*"man" + 0.027*"username" + 0.027*"old" + '
  '0.023*"utd" + 0.020*"trafford" + 0.020*"kane" + 0.019*"bruno" + '
  '0.018*"score"'),
 (5,
  '0.069*"username" + 0.016*"game" + 0.015*"pron" + 0.013*"one" + 0.013*"get" '
  '+ 0.012*"cavani" + 0.012*"back" + 0.011*"sign" +

In [52]:
modelseries3results.append(['model3_04',9,25,90,'symmetric',-6.2399,0.4592])

In [53]:
modeltracker3 = pd.DataFrame(modelseries3results, columns =['model_name', 'num_topics', 'passes','iterations','alpha','perplexity','coherence'], dtype = float) 
modeltracker = modeltracker.append(modeltracker3, ignore_index=True)
modeltracker

,model_name,num_topics,passes,iterations,alpha,perplexity,coherence
0,model1_01,4.0,5.0,50.0,symmetric,-6.5080,0.4128
1,model1_02,5.0,5.0,50.0,symmetric,-6.4341,0.4701
2,model1_03,6.0,5.0,50.0,symmetric,-6.4243,0.4459
3,model1_04,7.0,5.0,50.0,symmetric,-6.4231,0.4334
4,model1_05,8.0,5.0,50.0,symmetric,-6.3976,0.4446
5,model1_06,9.0,5.0,50.0,symmetric,-6.3812,0.4531
6,model1_07,10.0,5.0,50.0,symmetric,-6.3789,0.4203
7,model1_08,11.0,5.0,50.0,symmetric,-6.3670,0.4344
8,model1_09,12.0,5.0,50.0,symmetric,-6.3582,0.4226
9,model2_01,9.0,5.0,60.0,symmetric,-6.3813,0.4580


In [54]:
lda_model4_01 = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           workers=12,
                                           passes=15,
                                           random_state=1892,
                                           alpha='asymmetric',
                                           iterations=90,
                                           per_word_topics=True)
pprint(lda_model4_01.print_topics())
evaluator(lda_model4_01)

[(0,
  '0.029*"go" + 0.021*"penalty" + 0.017*"first" + 0.016*"maguire" + '
  '0.016*"game" + 0.016*"get" + 0.014*"goal" + 0.013*"man" + 0.012*"time" + '
  '0.010*"bruuunnooooo"'),
 (1,
  '0.058*"man" + 0.037*"utd" + 0.033*"red" + 0.029*"card" + 0.024*"martial" + '
  '0.021*"ole" + 0.013*"maguire" + 0.012*"penalty" + 0.012*"get" + '
  '0.011*"defend"'),
 (2,
  '0.043*"right" + 0.042*"play" + 0.040*"fan" + 0.037*"patrice" + 0.035*"evra" '
  '+ 0.035*"mulive" + 0.034*"team" + 0.033*"sky" + 0.032*"deserve" + '
  '0.032*"one"'),
 (3,
  '0.064*"live" + 0.040*"fee" + 0.038*"stream" + 0.036*"add" + 0.036*"telles" '
  '+ 0.035*"alex" + 0.034*"around" + 0.034*"player" + 0.033*"move" + '
  '0.032*"join"'),
 (4,
  '0.058*"goal" + 0.050*"son" + 0.033*"username" + 0.029*"man" + 0.027*"old" + '
  '0.026*"utd" + 0.023*"trafford" + 0.022*"kane" + 0.018*"mins" + '
  '0.017*"fernandes"'),
 (5,
  '0.085*"username" + 0.022*"game" + 0.020*"sign" + 0.020*"one" + '
  '0.016*"sancho" + 0.014*"pron" + 0.014*"ca

In [55]:
modeltracker4 = pd.DataFrame([['model4_01',9,15,90,'asymmetric',-6.2637,0.4461]], columns =['model_name', 'num_topics', 'passes','iterations','alpha','perplexity','coherence'], dtype = float)

In [56]:
modeltracker = modeltracker.append(modeltracker4, ignore_index=True)

In [57]:
modeltracker

,model_name,num_topics,passes,iterations,alpha,perplexity,coherence
0,model1_01,4.0,5.0,50.0,symmetric,-6.5080,0.4128
1,model1_02,5.0,5.0,50.0,symmetric,-6.4341,0.4701
2,model1_03,6.0,5.0,50.0,symmetric,-6.4243,0.4459
3,model1_04,7.0,5.0,50.0,symmetric,-6.4231,0.4334
4,model1_05,8.0,5.0,50.0,symmetric,-6.3976,0.4446
5,model1_06,9.0,5.0,50.0,symmetric,-6.3812,0.4531
6,model1_07,10.0,5.0,50.0,symmetric,-6.3789,0.4203
7,model1_08,11.0,5.0,50.0,symmetric,-6.3670,0.4344
8,model1_09,12.0,5.0,50.0,symmetric,-6.3582,0.4226
9,model2_01,9.0,5.0,60.0,symmetric,-6.3813,0.4580


# Final step

In [58]:
lda_model3_02.print_topics()

[(0,
  '0.047*"go" + 0.030*"first" + 0.019*"time" + 0.019*"goal" + 0.018*"game" + 0.016*"league" + 0.015*"need" + 0.015*"half" + 0.014*"let" + 0.013*"match"'),
 (1,
  '0.071*"man" + 0.049*"utd" + 0.049*"penalty" + 0.024*"get" + 0.018*"ole" + 0.013*"maguire" + 0.010*"player" + 0.009*"min" + 0.009*"defend" + 0.008*"need"'),
 (2,
  '0.043*"right" + 0.042*"fan" + 0.041*"play" + 0.037*"team" + 0.037*"patrice" + 0.034*"evra" + 0.034*"one" + 0.031*"deserve" + 0.031*"sky" + 0.030*"mulive"'),
 (3,
  '0.066*"live" + 0.039*"stream" + 0.037*"fee" + 0.034*"add" + 0.031*"telles" + 0.031*"alex" + 0.030*"join" + 0.030*"also" + 0.030*"player" + 0.030*"move"'),
 (4,
  '0.056*"goal" + 0.046*"son" + 0.031*"man" + 0.027*"old" + 0.023*"utd" + 0.022*"username" + 0.020*"trafford" + 0.020*"kane" + 0.017*"bruno" + 0.017*"score"'),
 (5,
  '0.075*"username" + 0.020*"game" + 0.014*"one" + 0.013*"get" + 0.013*"pron" + 0.012*"cavani" + 0.012*"back" + 0.012*"sign" + 0.010*"sancho" + 0.010*"see"'),
 (6,
  '0.034*"see"

In [59]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model3_02, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.394675 -0.084529       1        1  13.632041
4     -0.052937  0.142179       2        1  13.524504
1     -0.133461  0.109553       3        1  12.231572
5     -0.077654  0.002326       4        1  11.677344
0     -0.098568  0.073344       5        1  11.427720
8     -0.091299  0.129145       6        1  11.028007
6     -0.111110 -0.221126       7        1   9.372089
2     -0.086033 -0.259410       8        1   9.278449
3      0.256387  0.108518       9        1   7.828274, topic_info=         Term         Freq        Total Category  logprob  loglift
20     telles  3412.000000  3412.000000  Default  30.0000  30.0000
22       alex  2918.000000  2918.000000  Default  29.0000  29.0000
61        red  1981.000000  1981.000000  Default  28.0000  28.0000
27    medical  2008.000000  2008.000000  Default  27.0000  27.0000
157   martial  2322.000000  2322.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
20     telles   664.275639  3412.416993   Topic9  -3.4749   0.9109
60   username   331.390782  5493.114766   Topic9  -4.1703  -0.2605
84      watch   151.564552   897.208501   Topic9  -4.9526   0.7692
95     mulive   170.581693  1600.161714   Topic9  -4.8344   0.3088
163       man   108.905073  4654.171935   Topic9  -5.2831  -1.2076

[587 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
47        9  0.997375     accept
274       9  0.983043   accepted
272       1  0.938440     accord
272       3  0.034757     accord
48        3  0.002694        add
...     ...       ...        ...
1387      5  0.009081        wtf
1387      6  0.004541        wtf
1265      2  0.996255    yesssss
1265      7  0.001887    yesssss
6725      4  0.983763  zoom_call

[1471 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 2, 6, 1, 9, 7, 3, 4])

In [12]:
def topiclist(model, n):
    topiclist = []
    for i in range(0,n):
        modellist = [f'topic_{i+1}']
        words = []
        for i,j in model.show_topic(i):
            words.append(i)
        modellist.append(words)
        topiclist.append(modellist)
    return topiclist

In [13]:
modeldf = pd.DataFrame(topiclist(lda_model3_02, 9), columns = ['dominant topic', 'keywords'])

In [14]:
modeldf

,dominant topic,keywords
0,topic_1,"[go, first, time, game, goal, league, half, ne..."
1,topic_2,"[man, utd, penalty, ole, get, maguire, player,..."
2,topic_3,"[right, play, fan, patrice, team, evra, one, d..."
3,topic_4,"[live, stream, fee, add, telles, alex, join, a..."
4,topic_5,"[goal, son, man, username, old, utd, trafford,..."
5,topic_6,"[username, game, get, pron, one, cavani, sign,..."
6,topic_7,"[fan, see, sancho, buy, jose, mourinho, evra, ..."
7,topic_8,"[telles, alex, medical, hour, next, transfer, ..."
8,topic_9,"[martial, red, card, lamela, username, man, so..."


In [15]:
topics = lda_model3_02.get_document_topics(corpus)
topics_csr = gensim.matutils.corpus2csc(topics)
topics_numpy = topics_csr.T.toarray()
topics_df = pd.DataFrame(topics_numpy)
topics_df

,0,1,2,3,4,5,6,7,8
0,0.022224,0.022226,0.022225,0.022223,0.822209,0.022225,0.022223,0.022222,0.022223
1,0.037039,0.703686,0.037039,0.037044,0.037039,0.037039,0.037039,0.037039,0.037039
2,0.027781,0.027795,0.027789,0.027780,0.027790,0.777683,0.027790,0.027812,0.027780
3,0.055559,0.055589,0.055589,0.055565,0.555447,0.055574,0.055560,0.055557,0.055559
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.683077,0.000000,0.257080,0.000000
...,...,...,...,...,...,...,...,...,...
38696,0.037039,0.037040,0.037042,0.037039,0.037039,0.703685,0.037039,0.037039,0.037039
38697,0.015879,0.872948,0.015885,0.015880,0.015881,0.015890,0.015878,0.015877,0.015883
38698,0.055557,0.555522,0.055561,0.055566,0.055568,0.055558,0.055556,0.055556,0.055556
38699,0.022237,0.022255,0.822091,0.022246,0.022228,0.022245,0.022233,0.022226,0.022238


In [16]:
topics_df = topics_df.rename(columns={0:'topic_1', 1:'topic_2', 2:'topic_3', 3:'topic_4', 4:'topic_5', 5:'topic_6', 6:'topic_7', 7:'topic_8', 8:'topic_9'})

In [17]:
topics_df.idxmax(axis=1)

0        topic_5
1        topic_2
2        topic_6
3        topic_5
4        topic_6
          ...   
38696    topic_6
38697    topic_2
38698    topic_2
38699    topic_3
38700    topic_7
Length: 38701, dtype: object

In [18]:
df['dominanttopic'] = topics_df.idxmax(axis=1)

In [19]:
df.to_csv('../datasets/muntot_final.csv', index_label=False)

In [20]:
topiccount = pd.DataFrame(topics_df.idxmax(axis=1).value_counts())

In [21]:
modeldf = modeldf.join(topiccount, on='dominant topic').rename(columns = {0:'tweetcount'})

In [22]:
modeldf['topic_percent'] = modeldf['tweetcount'].apply(lambda x: x/38701)

In [23]:
modeldf

,dominant topic,keywords,tweetcount,topic_percent
0,topic_1,"[go, first, time, game, goal, league, half, ne...",4843,0.125139
1,topic_2,"[man, utd, penalty, ole, get, maguire, player,...",5805,0.149996
2,topic_3,"[right, play, fan, patrice, team, evra, one, d...",3317,0.085708
3,topic_4,"[live, stream, fee, add, telles, alex, join, a...",1926,0.049766
4,topic_5,"[goal, son, man, username, old, utd, trafford,...",6558,0.169453
5,topic_6,"[username, game, get, pron, one, cavani, sign,...",4847,0.125242
6,topic_7,"[fan, see, sancho, buy, jose, mourinho, evra, ...",3227,0.083383
7,topic_8,"[telles, alex, medical, hour, next, transfer, ...",2699,0.069740
8,topic_9,"[martial, red, card, lamela, username, man, so...",5479,0.141573


In [24]:
df[['dominanttopic','sentiment_rating','sentiment_score']].groupby('dominanttopic')['sentiment_score'].mean()

dominanttopic
topic_1    0.071517
topic_2   -0.088234
topic_3   -0.035481
topic_4    0.392808
topic_5    0.021831
topic_6    0.068779
topic_7    0.066595
topic_8    0.483549
topic_9   -0.046100
Name: sentiment_score, dtype: float64

In [25]:
modeldf.to_csv('../datasets/muntot_topics.csv', index_label=False)